In [ ]:
WITH 
	toDate('2025-03-30') AS my_date,
	full_table AS (SELECT
		sessions_st_mobile.report_date AS report_date,
	    installation_point_st_partner.partner_uuid AS partner_uuid,
	    installation_point_st_partner.installation_point_id  AS installation_point_id,
	    sessions_st_mobile.citizen_id AS citizen_id,
	    installation_point_st_partner.monetization AS monetization,
	    subscriptions_st_mobile.state AS subscriptions_state,
	    entries_st_mobile.ble_available AS ble_available,
	    citizens_st_mobile.activated_at AS activated_at,
	    city
	FROM db1.`sessions_st_mobile_ch` AS sessions_st_mobile
	LEFT JOIN db1.subscriptions_st_mobile_ch AS subscriptions_st_mobile 
	    ON subscriptions_st_mobile.citizen_id = sessions_st_mobile.citizen_id
	    AND subscriptions_st_mobile.report_date = sessions_st_mobile.report_date
	LEFT JOIN  (SELECT * FROM db1.`citizens_st_mobile_parquet_ch` WHERE report_date BETWEEN toStartOfInterval(my_date, INTERVAL 1 MONTH) AND my_date) AS citizens_st_mobile 
	    ON citizens_st_mobile.citizen_id = sessions_st_mobile.citizen_id
	    AND citizens_st_mobile.report_date = sessions_st_mobile.report_date
	LEFT JOIN db1.`flats_dir_partner_ch` AS flats_dir_partner ON flats_dir_partner.flat_uuid = citizens_st_mobile.flat_uuid
	LEFT JOIN db1.`entries_installation_points_dir_partner_ch` AS entries_installation_points_dir_partner ON flats_dir_partner.address_uuid = entries_installation_points_dir_partner.address_uuid
LEFT JOIN db1.`installation_point_st_partner_ch` AS installation_point_st_partner 
        ON installation_point_st_partner.installation_point_id = entries_installation_points_dir_partner.installation_point_id
        AND installation_point_st_partner.report_date = citizens_st_mobile.report_date
	LEFT JOIN db1.`entries_st_mobile_ch` AS entries_st_mobile
        ON `entries_st_mobile`.`report_date` = sessions_st_mobile.`report_date` 
        AND`entries_st_mobile`.`address_uuid` = flats_dir_partner.`address_uuid`
	WHERE toDate(`last_use`) BETWEEN toStartOfInterval(my_date, INTERVAL 1 MONTH) AND my_date
	   	 AND citizens_st_mobile.state = 'activated'
	)
SELECT
	my_date AS report_date,
    COUNT(DISTINCT citizen_id) AS total_active_users,
    COUNT(DISTINCT if(toDateOrNull(activated_at) BETWEEN toStartOfInterval(my_date, INTERVAL 1 MONTH) AND my_date ,citizen_id,Null)) AS new_active_users,
	COUNT(DISTINCT if(monetization = 1, citizen_id, Null)) as total_active_users_monetization,
	COUNT(DISTINCT if(monetization = 1 and subscriptions_state = 'activated',citizen_id,Null)) as total_active_user_subscribed_monetization,
	COUNT(DISTINCT if(ble_available = 'true',citizen_id, Null)) AS totalmy_date_active_users_ble_available,
	COUNT(DISTINCT if(ble_available = 'true' and monetization = 1,citizen_id, Null)) AS total_active_users_ble_available_monetization,
    COUNT(DISTINCT if(ble_available = 'true' and monetization = 1 and subscriptions_state = 'activated',citizen_id, Null)) AS total_active_users_ble_available_subscribed_monetization
FROM full_table
LIMIT 10